# 2D Crack propagation in a bended beam

 - @todo: Plot the residual value
 - @todo: State dependencies - recalculate upon material parameter change

\begin{align}
\sigma_\mathrm{F} = E w / L_\mathrm{eff}
\end{align}

In [80]:
%matplotlib widget
import numpy as np
import matplotlib.pylab as plt
from bmcs_cross_section.api import ReinfLayer
import bmcs_utils.api as bu
from bmcs_shear.shear_crack.crack_propagation import CrackPropagation
ce = CrackPropagation(n_seg=100)
#ce.sz_bd.cmm.trait_set(tau_1 = 4, s_1 = 0.2, tau_2 = 3, tau_3 = 2, s_2 = 1.4, s_3 = 5)
ce.sz_bd.cmm.trait_set(f_c = 32.9, f_t = 0.3 * 32.9 **(2/3), d_a = 16)
ce.sz_bd.Rectangle = True
ce.sz_bd.trait_set(H=600, B=250, L=3150)
ce.sz_cp.trait_set(x_00=ce.sz_bd.L * 0.1, n_m = 4)
ce.sz_cp.sz_ctr.L_fps = 80
ce.sz_cp.sz_ctr.trait_set(ell = 2)
ce.sz_bd.smm.trait_set(d_s = 28, n = 2)
bl1 = ReinfLayer(name='layer1', z=44, A=ce.sz_bd.smm.A_f, matmod='steel')
bl1.matmod_.trait_set(E=ce.sz_bd.smm.E_f, f_t=ce.sz_bd.smm.sig_y)
ce.sz_bd.cross_section_layout.add_layer(bl1)
ce.sz_bd
#F_La, F_Na, F_m, Q_reduce, c_p = ce.analyze() 
#print(F_La, F_Na, c_p)

In [81]:
# bu.InteractiveWindow([ce, ce.sz_bd, ce.sz_bd.cross_section_layout, 
#                       ce.sz_stress_profile, 
#                       ce.sz_bd.cmm, ce.sz_bd.smm, ce.sz_cp, ce.sz_cp.sz_ctr]).interact()
ce.interact()

<lambdifygenerated-332>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))


<lambdifygenerated-332>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))


In [11]:
ce.sz_stress_profile.F_Na

array([[1.15412003e+05, 2.99848142e+00]])

In [ ]:
import numpy as np
crack_position = 0.1 * ce.sz_bd.L
ce.reset()
ce.sz_cp.trait_set(x_00=crack_position)
ce.sz_cp.sz_ctr.L_fps = 80
ce.run()
F_beam = ce.F_beam
F_max_ = np.max(F_beam)
F_max_i_ = np.argmax(F_beam)
F_max_i_int = np.array(F_max_i_)
F_Na_ = ce.F_Na[F_max_i_int]
F_s_ = np.array(ce.F_s)
w = ce.w_steel[F_max_i_int]
F_max_
F_s_

In [ ]:
ce.reset()
ini_crck_point = ce.sz_cp.trait_set(x_00= 0.1 * ce.sz_bd.L)
ce.run()
H_s = []
F_beam = ce.F_beam
F_max_ = np.max(F_beam)
F_max_i_ = np.argmax(F_beam)
F_max_i_int = np.array(F_max_i_)
x_tip_1k_ = ce.x_tip_1k[F_max_i_int]
for i in range(len(F_beam)):
    if ce.x_tip_1k[i] <= x_tip_1k_:
        H_s_ = ce.sz_bd.H - ce.x_tip_1k[i]
        #shear_agg = ce.shear_agg[i]
        H_s.append(H_s_)
    else:
        break
print(H_s)
    #print(shear_agg)
# H_s = []
# F_beam = ce.F_beam
# F_max_ = np.max(F_beam)
# for i in range(len(F_beam)):
#     if F_beam[i] == F_max_:
#         H_s = ce.H_s
#         #H_s.append(H_s)
#     else:
#         break
#     print(H_s)

In [ ]:
import matplotlib.pylab as plt
_, (ax1) = plt.subplots(1, 1, figsize=(8,4))
ax1.plot(H_s, F_s_ / 1000, color = 'blue', lw= 2, label = '$\mathrm{F_{s}}$');
ax1.set_xlabel(r'$\mathrm{Height\,of\,crack\,(mm)}$'); ax1.set_ylabel(r'$\mathrm{F_s\,(kN)}$');
ax1.set_title(r'Force at steel reinforcement')
ax1.legend()
plt.savefig("D:\Shear zones\Python plots\F_steel.pdf")

In [ ]:
#w, F_s_, F_max_

In [ ]:
import numpy as np
crack_position = 0.1 * ce.sz_bd.L
ce.reset()
ce.sz_cp.trait_set(x_00=crack_position)
ce.sz_cp.sz_ctr.L_fps = 80
ce.run()
F_beam = ce.F_beam
F_max_ = np.max(F_beam)
F_max_i_ = np.argmax(F_beam)
F_max_i_int = np.array(F_max_i_)
F_Na_ = ce.F_Na[F_max_i_int]
F_s_ = ce.F_s[F_max_i_int]
slip_ = ce.slip[F_max_i_int]
w_ = ce.w[F_max_i_int][0]
steel_slip_ = ce.s_steel[F_max_i_int]
w_steel_ = ce.w_steel[F_max_i_int]
shear_stress_ = ce.shear_agg[F_max_i_int]
F_a_ = ce.F_a[F_max_i_int][1]
F_agg_ = F_a_ - F_Na_
# shear_stress_, slip_

In [ ]:
import matplotlib.pylab as plt
_, (ax1) = plt.subplots(1, 1, figsize=(8,4))
s = np.linspace(0,0.3,100)
V_df = ce.sz_bd.smm.get_V_df(s)
ax1.plot(s, V_df , color = 'blue', lw= 2, label = '$V_{da}$');
ax1.set_xlabel(r'$slip\,(mm)}$'); ax1.set_ylabel(r'$V (N)$');
ax1.set_title(r'Dowel Force Contribution')
ax1.legend()
plt.savefig("D:\Shear zones\Python plots\Dowel_force.pdf")

In [ ]:
# import numpy as np
# ce.sz_stress_profile.u_La
# idx = np.argmax(ce.F_beam)
# #v_n = ce.v_n
# #v_n[idx]
# #ce.F_Na[idx]
# idx

In [ ]:
# ce.sz_stress_profile.u_Na

In [ ]:
# ce.reset()
# ini_crck_point = ce.sz_cp.trait_set(x_00= 0.1 * ce.sz_bd.L)
# ce.run()
# F_beam = ce.F_beam
# F_max_ = np.max(F_beam)
# for i in range(len(F_beam)):
#     if F_beam[i] < F_max_:
#         pass
#     elif F_beam[i] == F_max_:
#         slip = ce.slip[i]
#         #H_s = ce.H_s
#         shear_agg = ce.shear_agg[i]
#         print('shear agg', shear_agg)
#         print('slip', slip)
#         #print(shear_agg)
#         #print(H_s)

In [ ]:
# ce.reset()
# ini_crck_point = ce.sz_cp.trait_set(x_00= 0.9 * ce.sz_bd.L)
# ce.run()
# F_beam = ce.F_beam
# F_max_ = np.max(F_beam)
# F_max_idx = np.argmax(F_beam)
# F_Na_ = ce.F_Na[F_max_idx]
# s_s = ce.s_steel[:F_max_idx]
# V_da = ce.F_Na[:F_max_idx]
# slip = ce.slip
# shear_agg = ce.shear_agg

In [ ]:
# import matplotlib.pylab as plt
# _, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
# ax1.plot(s_s, V_da , color = 'blue', lw= 2, label = '$V_{da}$');
# ax1.set_xlabel(r'$slip\,(mm)}$'); ax1.set_ylabel(r'$V (N)$');
# ax1.set_title(r'Dowel Force Contribution')
# ax1.legend()
# ax2.plot(slip, shear_agg , color = 'red', lw= 2, label = r'$\tau_{agg}$');
# ax2.set_xlabel(r'$slip\,(mm)}$'); ax2.set_ylabel(r'$\tau_{agg}$');
# ax2.set_title(r'Aggregate Force Contribution at Ultimate state')
# ax2.legend()
# #plt.savefig("D:\Shear zones\Python plots\dowel_agg_09.pdf")

In [ ]:
# import matplotlib.pylab as plt
# _, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
# ax1.plot(ce.sz_stress_profile.u_Lb[:,1], ce.sz_stress_profile.S_Lb[:,1] , color = 'blue', lw= 2, label = '$F_{agg}$');
# ax2.plot(s_s, ce.sz_stress_profile.S_Lb[:,0] , color = 'blue', lw= 2, label = '$F_{agg}$');

In [ ]:
# import numpy as np
# np.max(ce.F_beam)# + ce.crack_tip_shear_stress.Q + ce.crack_tip_shear_stress.Q_reduced

In [ ]:
#ce.sz_stress_profile.F_La

In [ ]:
import numpy as np
crack_position = 0.1 * ce.sz_bd.L
ce.reset()
ce.sz_cp.trait_set(x_00=crack_position)
ce.sz_cp.sz_ctr.L_fps = 80
ce.run()
F_beam = ce.F_beam
F_max_ = np.max(F_beam)
F_max_i_ = np.argmax(F_beam)
F_max_i_int = np.array(F_max_i_)
F_Na_ = ce.F_Na[F_max_i_int]
F_s_ = ce.F_s[F_max_i_int]
M = ce.M
slip_ = ce.slip[F_max_i_int]
w_ = ce.w[F_max_i_int][0]
steel_slip_ = ce.s_steel[F_max_i_int]
w_steel_ = ce.w_steel
shear_stress_ = ce.shear_agg[F_max_i_int]
F_a_ = ce.F_a[F_max_i_int][1]
F_agg_ = F_a_ - F_Na_
#shear_stress_, slip_
#M, w_steel_[F_max_i_int]

In [ ]:
import matplotlib.pylab as plt
E_s = 210000
A_s = 2 * 3.14 * 28**2 / 4
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(np.array(M[1:]) / (E_s * A_s), w_steel_[1:] , color = 'blue', lw= 2); #/1000#
#ax1.plot(slip, , color = 'red', lw = 2, label = '$F_{dowel}$');
ax1.set_xlabel(r'$\frac{M}{E_s A_s}$'); ax1.set_ylabel(r'$w_{steel}\,(mm)$');
ax1.set_title(r'Bending moment crack opening relationship')
ax1.legend()
plt.savefig("D:\Shear zones\Python plots\co.pdf")

In [ ]:
import numpy as np
crack_positions = np.linspace(0.1 * ce.sz_bd.L, 0.9 * ce.sz_bd.L, 3)
F_max = []
F_a = []
Q = []
Q_red = []
F_max_i = []
F_Na = []
F_s = []
slip = []
steel_slip = []
shear_stress = []
w = []
w_steel = []
V_c = []
F_agg = []
for crack_position in crack_positions:
    ce.reset()
    ce.sz_cp.trait_set(x_00=crack_position)
    ce.sz_cp.sz_ctr.L_fps = 80
    ce.run()
    F_beam = ce.F_beam
    F_max_ = np.max(F_beam)
    F_max_i_ = np.argmax(F_beam)
    F_max_i_int = np.array(F_max_i_)
    F_Na_ = ce.F_Na[F_max_i_int]
    F_s_ = ce.F_s[F_max_i_int]
    slip_ = ce.slip[F_max_i_int][0]
    w_ = ce.w[F_max_i_int][0]
    steel_slip_ = ce.s_steel[F_max_i_int]
    w_steel_ = ce.w_steel[F_max_i_int]
    shear_stress_ = ce.shear_agg[F_max_i_int][0]
    F_a_ = ce.F_a[F_max_i_int][1]
    F_agg_ = F_a_ - F_Na_
    Q_ = ce.Q[F_max_i_int]
    Q_red_ =  ce.Q_red[F_max_i_int]
    #V_c_ = Q_ + Q_red_ + F_a_
    F_max.append(F_max_)
    F_max_i.append(F_max_i_)
    F_Na.append(F_Na_)
    F_s.append(F_s_)
    F_a.append(F_a_)
    F_agg.append(F_agg_)
    slip.append(slip_)
    w.append(w_)
    steel_slip.append(steel_slip_)
    w_steel.append(w_steel_)
    shear_stress.append(shear_stress_)
    Q.append(Q_)
    Q_red.append(Q_red_)
    #V_c.append(V_c_)
# print('F_max =', F_max)
# print('F_max_i =', F_max_i)
# print('F_Na =', F_Na)
# print('F_s =', F_s)
# #print(type(F_Na))
# print('F_a = ', F_a)
# print('Q =', Q)
# print('Q_red =', Q_red)
# # #print('V_c =', V_c)
print('crack position =', crack_positions)
print('F_max',  F_max)
print('F_agg', F_agg)
print('F_Na', F_Na)
#print('slip =', slip)
#print('shear stress =', shear_stress)
#print('steel slip =', steel_slip)
#print('F_agg =', F_agg)

In [ ]:
#F_a

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(crack_positions, F_agg , color = 'blue', lw= 2, label = r'$F_{agg}$'); #/1000#
#ax1.plot(slip, , color = 'red', lw = 2, label = '$F_{dowel}$');
ax1.set_xlabel(r'$\mathrm{Initial\,position\,of\,the\,crack\,(mm)}$'); ax1.set_ylabel(r'$F\,(N)$');
ax1.set_title(r'Force contribution due to aggregate interlock')
ax1.legend()
ax2.plot(crack_positions, F_max, color = 'blue', lw= 2, label = r'$F_{agg}$'); 
ax2.set_xlabel(r'$\mathrm{Initial\,position\,of\,the\,crack\,(mm)}$'); ax2.set_ylabel(r'$F\,(N)$');
ax2.set_title(r'Shear Capacity')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\max_force_shear_force_agg_Lfps2.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(crack_positions, shear_stress , color = 'blue', lw= 2, label = r'$\tau_{agg}$'); #/1000#
#ax1.plot(slip, , color = 'red', lw = 2, label = '$F_{dowel}$');
ax1.set_xlabel(r'$\mathrm{Initial\,position\,of\,the\,crack\,(mm)}$'); ax1.set_ylabel(r'$\tau\,(MPa)$');
ax1.set_title(r'Shear Stress due to Aggregate Interlock')
ax1.legend()
ax2.plot(crack_positions, slip , color = 'red', lw= 2, label = '$slip\,(mm)$');
ax2.set_xlabel(r'$\mathrm{Initial\,position\,of\,the\,crack\,(mm)}$'); ax2.set_ylabel(r'$slip\,(mm)$');
ax2.set_title(r'Slip due to Aggregate Interlock')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\slip_shear_Lfps2.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(crack_positions,  F_Na, color = 'blue', lw= 2, label = r'$F_{Na}$'); #/1000#
ax1.set_xlabel(r'$\mathrm{Initial\,position\,of\,the\,crack\,(mm)}$'); ax1.set_ylabel(r'$F\,(N)$');
ax1.set_title(r'Dowel force due to Dowel Action')
ax2.plot(crack_positions, steel_slip , color = 'red', lw= 2, label = '$slip\,(mm)$');
ax2.set_xlabel(r'$\mathrm{Initial\,position\,of\,the\,crack\,(mm)}$'); ax2.set_ylabel(r'$steel\,slip\,(mm)$');
ax2.set_title(r'Slip due to dowel force')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\dowel_slip_length_Lfps2.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(crack_positions, w, color = 'blue', lw= 2, label = r'$w(mm)}$'); #/1000#
ax1.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax1.set_ylabel(r'$w\,(mm)$');
ax1.set_title(r'Crack opening at the ultimate state due to aggregate')
ax1.legend()
ax2.plot(crack_positions, w_steel , color = 'red', lw= 2, label = '$w\,(mm)$');
ax2.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax2.set_ylabel(r'$w\,(mm)$');
ax2.set_title(r'Crack Opening at the ultimate state at longitudinal reinforcement')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\w_steel_Lfps2.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(crack_positions, np.array(F_max) / 1000, lw= 2, label = r'$\mathrm{F_{max}}$'); #/1000#
ax1.set_xlabel(r'$\mathrm{Position\,of\,single\,crack\,(mm)}$'); ax1.set_ylabel(r'$F\,(kN)$');
ax1.set_title(r'Maximum capacity and force contributions')
ax1.legend()
ax2.plot(crack_positions, np.array(F_agg) / 1000, lw= 2, label = r'$\mathrm{F_{agg}}$');
ax2.plot(crack_positions, np.array(F_Na) / 1000, lw= 2, label = r'$\mathrm{F_{Na}}$');
ax2.legend()
#ax2.plot(crack_positions, np.array(Q_red) / 1000, lw= 2, label = r'$\mathrm{F_{uncr}}$');
plt.savefig("D:\Shear zones\Python plots\contributions_conference.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1) = plt.subplots(1, 1, figsize=(11,4))
ax1.plot(crack_positions, F_s, color = 'red', lw = 2, label = '$F_{s}$');
ax1.set_xlabel(r'$initial\,crack\,position\,(mm)$'); ax1.set_ylabel(r'$F_{steel}\,(kN)$');
ax1.legend()
plt.savefig("D:\Shear zones\Python plots\F_s.pdf")

In [ ]:
L_fps_ = np.linspace(2, 80, 10)
crack_position = ce.sz_bd.L  * 0.5
F_max = []
F_max_i = []
F_a = []
F_Na = []
F_s = []
slip = []
steel_slip = []
shear_stress = []
w = []
w_steel = []
F_agg = []
for L_fps in L_fps_:
    ce.reset()
    ce.sz_cp.sz_ctr.trait_set(L_fps=L_fps)
    ce.sz_cp.trait_set(x_00=crack_position)
    ce.run()
    F_beam = ce.F_beam
    F_max_ = np.max(F_beam)
    F_max_i_ = np.argmax(F_beam)
    F_max_i_int = np.array(F_max_i_)
    F_Na_ = ce.F_Na[F_max_i_int]
    F_s_ = ce.F_s[F_max_i_int]
    slip_ = ce.slip[F_max_i_int][0]
    w_ = ce.w[F_max_i_int][0]
    steel_slip_ = ce.s_steel[F_max_i_int]
    w_steel_ = ce.w_steel[F_max_i_int]
    shear_stress_ = ce.shear_agg[F_max_i_int][0]
    F_a_ = ce.F_a[F_max_i_int][1]
    F_agg_ = F_a_ - F_Na_
#     F_beam = ce.F_beam
#     F_max_ = np.max(F_beam)
#     F_max_i_ = np.argmax(F_beam)
#     F_max_i_int = np.array(F_max_i_)
    F_max_i.append(F_max_i_)
    F_max.append(F_max_)
    F_Na.append(F_Na_)
    F_s.append(F_s_)
    F_a.append(F_a_)
    F_agg.append(F_agg_)
    slip.append(slip_)
    w.append(w_)
    steel_slip.append(steel_slip_)
    w_steel.append(w_steel_)
    shear_stress.append(shear_stress_)
#print(F_max)
#print(F_max_i)
print('slip =', slip)
print('w =', w)
print('w_steel =', w_steel)
print('steel_slip =', steel_slip)
print('L_fps =', L_fps_)

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(L_fps_, steel_slip, color = 'blue', lw= 2, label = r'$s_{s}(mm)}$');#/1000#
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax1.set_ylabel(r'$s\,(mm)$');
ax1.set_title(r'Slip at the steel reinforcement $x_{00}$= 0.5L')
ax1.legend()
ax2.plot(L_fps_, w_steel , color = 'red', lw= 2, label = '$w\,(mm)$');
ax2.set_xlabel('$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax2.set_ylabel(r'$w\,(mm)$');
ax2.set_title(r'Crack Opening at the ultimate state at longitudinal reinforcement')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\w_steel_Lfps_vary.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(L_fps_,F_Na, color = 'blue', lw= 2);#/1000#
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax1.set_ylabel(r'$F_{da}\,(Na)$');
ax1.set_title(r'Dowel Force')
ax1.legend()
ax2.plot(L_fps_, F_a , color = 'red', lw= 2);
ax2.set_xlabel('$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax2.set_ylabel(r'$F_{ag}\,(N)$');
ax2.set_title(r'Aggregate Force')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\F_Na_agg_vary.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(L_fps_, slip, color = 'blue', lw= 2, label = r'$s(mm)}$');#/1000#
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax1.set_ylabel(r'$s\,(mm)$');
ax1.set_title(r'Slip at aggregate interlock')
ax1.legend()
ax2.plot(L_fps_, shear_stress , color = 'red', lw= 2, label = '$\tau\,(MPa)$');
ax2.set_xlabel('$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax2.set_ylabel(r'$\tau_{ag}\,(MPa)$');
ax2.set_title(r'Shear stress due to aggregate interlock')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\shear_slip_agg_vary.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(L_fps_, F_max, color = 'blue', lw= 2, label = r'$F(N)}$');#/1000#
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax1.set_ylabel(r'$F_{max}\,(N)$');
ax1.set_title(r'Maximum Force Contribution')
ax1.legend()
ax2.plot(L_fps_, F_s , color = 'red', lw= 2, label = '$F_s\,(mm)$');
ax2.set_xlabel('$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax2.set_ylabel(r'$F_{s}\,(N)$');
ax2.set_title(r'Horizontal Force component at steel')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\F_max_F_steel_vary.pdf")

In [ ]:
L_fps_num = 4
L_fps_ = np.linspace(2, 80, L_fps_num)
crack_positions_num = 9
crack_positions = np.linspace(ce.sz_bd.L  * 0.1, ce.sz_bd.L  * 0.9, crack_positions_num) 
# F_max = []
F_max = np.zeros([crack_positions_num, L_fps_num])
F_Na = np.zeros([crack_positions_num, L_fps_num])
F_s = np.zeros([crack_positions_num, L_fps_num])
F_a = np.zeros([crack_positions_num, L_fps_num])
slip = np.zeros([crack_positions_num, L_fps_num])
steel_slip = np.zeros([crack_positions_num, L_fps_num])
shear_stress = np.zeros([crack_positions_num, L_fps_num])
w = np.zeros([crack_positions_num, L_fps_num])
w_steel = np.zeros([crack_positions_num, L_fps_num])
F_agg = np.zeros([crack_positions_num, L_fps_num])
for i in range(len(crack_positions)):
    for j in range(len(L_fps_)):
        ce.reset()
        ce.sz_cp.trait_set(x_00=crack_positions[i])
        print(crack_positions[i])
        ce.sz_cp.sz_ctr.trait_set(L_fps=L_fps_[j])
        #print(L_fps_[j])
        ce.run()
        F_beam = ce.F_beam
        F_max_ = np.max(F_beam)
        F_max_i_ = np.argmax(F_beam)
        F_max_i_int = np.array(F_max_i_)
        F_Na_ = ce.F_Na[F_max_i_int]
        F_s_ = ce.F_s[F_max_i_int]
        slip_ = ce.slip[F_max_i_int][0]
        w_ = ce.w[F_max_i_int][0]
        steel_slip_ = ce.s_steel[F_max_i_int]
        w_steel_ = ce.w_steel[F_max_i_int]
        shear_stress_ = ce.shear_agg[F_max_i_int][0]
        F_a_ = ce.F_a[F_max_i_int][1]
        F_agg_ = F_a_ - F_Na_
        F_max[i, j] = F_max_
        F_Na[i,j] = F_Na_
        F_s[i,j] = F_s_
        slip[i,j] = slip_
        w[i,j] = w_
        steel_slip[i,j] = steel_slip_
        w_steel[i,j] = w_steel_
        shear_stress[i,j] = shear_stress_
        F_a[i,j] = F_a_
        F_agg[i,j] = F_agg_
        
        
#         F_max_i.append(F_max_i_)
    #F_max_all = np.concatenate(F_max_1)
print('F_max =', F_max)
print('F_Na =', F_Na)
print('F_s =', F_s)
print('slip =', slip)
print('w =', w)
print('steel_slip =', steel_slip)
print('w_steel =', w_steel)
print('shear_stress =', shear_stress)
print('F_a =', F_a)
    #r.append(F_max)
#         F_Na.append(F_Na_)
#         F_s.append(F_s_)
#         F_a.append(F_a_)
#         F_agg.append(F_agg_)
#         #slip.append(slip_)
#         #w.append(w_)
#         steel_slip.append(steel_slip_)
#         w_steel.append(w_steel_)
        #shear_stress.append(shear_stress_)
    #print('slip =', slip)
    #print('w =', w)
#     print('steel_slip =', steel_slip)
#     print('L_fps =', L_fps_)
#     print('crack_position =', crack_positions)

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(L_fps_, F_max[0,:], lw= 2, label = r'$x_{00} = 0.1L$');
ax1.plot(L_fps_, F_max[1,:], lw= 2, label = r'$x_{00} = 0.2L$');
ax1.plot(L_fps_, F_max[2,:], lw= 2, label = r'$x_{00} = 0.3L$');
ax1.plot(L_fps_, F_max[3,:], lw= 2, label = r'$x_{00} = 0.4L$');
ax1.plot(L_fps_, F_max[4,:], lw= 2, label = r'$x_{00} = 0.5L$');
ax1.plot(L_fps_, F_max[5,:], lw= 2, label = r'$x_{00} = 0.6L$');
ax1.plot(L_fps_, F_max[6,:], lw= 2, label = r'$x_{00} = 0.7L$');
ax1.plot(L_fps_, F_max[7,:], lw= 2, label = r'$x_{00} = 0.8L$');
ax1.plot(L_fps_, F_max[8,:], lw= 2, label = r'$x_{00} = 0.9L$');
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax1.set_ylabel(r'$F_{max}\,(N)$');
ax1.set_title(r'Maximum Force Contribution')
ax1.legend()
ax2.plot(L_fps_, F_Na[0,:], lw= 2, label = r'$x_{00} = 0.1L$');
ax2.plot(L_fps_, F_Na[1,:], lw= 2, label = r'$x_{00} = 0.2L$');
ax2.plot(L_fps_, F_Na[2,:], lw= 2, label = r'$x_{00} = 0.3L$');
ax2.plot(L_fps_, F_Na[3,:], lw= 2, label = r'$x_{00} = 0.4L$');
ax2.plot(L_fps_, F_Na[4,:], lw= 2, label = r'$x_{00} = 0.5L$');
ax2.plot(L_fps_, F_Na[5,:], lw= 2, label = r'$x_{00} = 0.6L$');
ax2.plot(L_fps_, F_Na[6,:], lw= 2, label = r'$x_{00} = 0.7L$');
ax2.plot(L_fps_, F_Na[7,:], lw= 2, label = r'$x_{00} = 0.8L$');
ax2.plot(L_fps_, F_Na[8,:], lw= 2, label = r'$x_{00} = 0.9L$');
ax2.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax2.set_ylabel(r'$F_{Na}\,(N)$');
ax2.set_title(r'Maximum Dowel Force Contribution')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\F_max_F_steel_all_crack_points.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(L_fps_, F_s[0,:], lw= 2, label = r'$x_{00} = 0.1L$');
ax1.plot(L_fps_, F_s[1,:], lw= 2, label = r'$x_{00} = 0.2L$');
ax1.plot(L_fps_, F_s[2,:], lw= 2, label = r'$x_{00} = 0.3L$');
ax1.plot(L_fps_, F_s[3,:], lw= 2, label = r'$x_{00} = 0.4L$');
ax1.plot(L_fps_, F_s[4,:], lw= 2, label = r'$x_{00} = 0.5L$');
ax1.plot(L_fps_, F_s[5,:], lw= 2, label = r'$x_{00} = 0.6L$');
ax1.plot(L_fps_, F_s[6,:], lw= 2, label = r'$x_{00} = 0.7L$');
ax1.plot(L_fps_, F_s[7,:], lw= 2, label = r'$x_{00} = 0.8L$');
ax1.plot(L_fps_, F_s[8,:], lw= 2, label = r'$x_{00} = 0.9L$');
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax1.set_ylabel(r'$F_{s}\,(N)$');
ax1.set_title(r'Maximum Horizontal Force Contribution at reinforcement')
ax1.legend()
ax2.plot(L_fps_, F_a[0,:], lw= 2, label = r'$x_{00} = 0.1L$');
ax2.plot(L_fps_, F_a[1,:], lw= 2, label = r'$x_{00} = 0.2L$');
ax2.plot(L_fps_, F_a[2,:], lw= 2, label = r'$x_{00} = 0.3L$');
ax2.plot(L_fps_, F_a[3,:], lw= 2, label = r'$x_{00} = 0.4L$');
ax2.plot(L_fps_, F_a[4,:], lw= 2, label = r'$x_{00} = 0.5L$');
ax2.plot(L_fps_, F_a[5,:], lw= 2, label = r'$x_{00} = 0.6L$');
ax2.plot(L_fps_, F_a[6,:], lw= 2, label = r'$x_{00} = 0.7L$');
ax2.plot(L_fps_, F_a[7,:], lw= 2, label = r'$x_{00} = 0.8L$');
ax2.plot(L_fps_, F_a[8,:], lw= 2, label = r'$x_{00} = 0.9L$');
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax2.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax2.set_ylabel(r'$F_{ag}\,(N)$');
ax2.set_title(r'Maximum Aggregate Force Contribution at Ultimate state')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\F_a_F_steel_h_all_crack_points.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(L_fps_, steel_slip[0,:], lw= 2, label = r'$x_{00} = 0.1L$');
ax1.plot(L_fps_, steel_slip[1,:], lw= 2, label = r'$x_{00} = 0.2L$');
ax1.plot(L_fps_, steel_slip[2,:], lw= 2, label = r'$x_{00} = 0.3L$');
ax1.plot(L_fps_, steel_slip[3,:], lw= 2, label = r'$x_{00} = 0.4L$');
ax1.plot(L_fps_, steel_slip[4,:], lw= 2, label = r'$x_{00} = 0.5L$');
ax1.plot(L_fps_, steel_slip[5,:], lw= 2, label = r'$x_{00} = 0.6L$');
ax1.plot(L_fps_, steel_slip[6,:], lw= 2, label = r'$x_{00} = 0.7L$');
ax1.plot(L_fps_, steel_slip[7,:], lw= 2, label = r'$x_{00} = 0.8L$');
ax1.plot(L_fps_, steel_slip[8,:], lw= 2, label = r'$x_{00} = 0.9L$');
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax1.set_ylabel(r'$s\,(mm)$');
ax1.set_title(r'Slip displacement at steel reinforcement')
ax1.legend()
ax2.plot(L_fps_, w_steel[0,:], lw= 2, label = r'$x_{00} = 0.1L$');
ax2.plot(L_fps_, w_steel[1,:], lw= 2, label = r'$x_{00} = 0.2L$');
ax2.plot(L_fps_, w_steel[2,:], lw= 2, label = r'$x_{00} = 0.3L$');
ax2.plot(L_fps_, w_steel[3,:], lw= 2, label = r'$x_{00} = 0.4L$');
ax2.plot(L_fps_, w_steel[4,:], lw= 2, label = r'$x_{00} = 0.5L$');
ax2.plot(L_fps_, w_steel[5,:], lw= 2, label = r'$x_{00} = 0.6L$');
ax2.plot(L_fps_, w_steel[6,:], lw= 2, label = r'$x_{00} = 0.7L$');
ax2.plot(L_fps_, w_steel[7,:], lw= 2, label = r'$x_{00} = 0.8L$');
ax2.plot(L_fps_, w_steel[8,:], lw= 2, label = r'$x_{00} = 0.9L$');
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax2.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax2.set_ylabel(r'$w\,(mm)$');
ax2.set_title(r'Longitudinal crack opening at steel reinforcement')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\crack_position_l_2_newcrack_position_l_2_new.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
ax1.plot(L_fps_, slip[0,:], lw= 2, label = r'$x_{00} = 0.1L$');
ax1.plot(L_fps_, slip[1,:], lw= 2, label = r'$x_{00} = 0.2L$');
ax1.plot(L_fps_, slip[2,:], lw= 2, label = r'$x_{00} = 0.3L$');
ax1.plot(L_fps_, slip[3,:], lw= 2, label = r'$x_{00} = 0.4L$');
ax1.plot(L_fps_, slip[4,:], lw= 2, label = r'$x_{00} = 0.5L$');
ax1.plot(L_fps_, slip[5,:], lw= 2, label = r'$x_{00} = 0.6L$');
ax1.plot(L_fps_, slip[6,:], lw= 2, label = r'$x_{00} = 0.7L$');
ax1.plot(L_fps_, slip[7,:], lw= 2, label = r'$x_{00} = 0.8L$');
ax1.plot(L_fps_, slip[8,:], lw= 2, label = r'$x_{00} = 0.9L$');
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax1.set_ylabel(r'$s\,(mm)$');
ax1.set_title(r'Slip displacement at interlock at ultimate state')
ax1.legend()
ax2.plot(L_fps_, shear_stress[0,:], lw= 2, label = r'$x_{00} = 0.1L$');
ax2.plot(L_fps_, shear_stress[1,:], lw= 2, label = r'$x_{00} = 0.2L$');
ax2.plot(L_fps_, shear_stress[2,:], lw= 2, label = r'$x_{00} = 0.3L$');
ax2.plot(L_fps_, shear_stress[3,:], lw= 2, label = r'$x_{00} = 0.4L$');
ax2.plot(L_fps_, shear_stress[4,:], lw= 2, label = r'$x_{00} = 0.5L$');
ax2.plot(L_fps_, shear_stress[5,:], lw= 2, label = r'$x_{00} = 0.6L$');
ax2.plot(L_fps_, shear_stress[6,:], lw= 2, label = r'$x_{00} = 0.7L$');
ax2.plot(L_fps_, shear_stress[7,:], lw= 2, label = r'$x_{00} = 0.8L$');
ax2.plot(L_fps_, shear_stress[8,:], lw= 2, label = r'$x_{00} = 0.9L$');
#ax1.plot(crack_positions, slip, color = 'red', lw= 2, label = r'$s(mm)}$');
ax2.set_xlabel(r'$\mathrm{Length\,of\,FPZ\,(mm)}$'); ax2.set_ylabel(r'$\tau\,(MPa)$');
ax2.set_title(r'shear stress distribution due to aggregate interlock')
ax2.legend()
plt.savefig("D:\Shear zones\Python plots\slip_shear_agg_all_crack_points.pdf")

In [ ]:
import matplotlib.pylab as plt
_, (ax1) = plt.subplots(1, 1, figsize=(6,4))
ax1.plot(crack_positions, F_s[:, 1], lw= 2, label = r'$F_s$');
ax1.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax1.set_ylabel(r'$F_{s}\,(N)$');
ax1.set_title(r'Maximum Horizontal Force Contribution at reinforcement')
ax1.legend()
plt.savefig("D:\Shear zones\Python plots\F_s_length_beam.pdf")

In [ ]:
# seg_num = 4
# seg_ = np.linspace(40, 140, seg_num)
# crack_positions_num = 9
# crack_positions = np.linspace(ce.sz_bd.L  * 0.1, ce.sz_bd.L  * 0.9, crack_positions_num) 
# # F_max = []
# F_max = np.zeros([seg_num, crack_positions_num])
# F_Na = np.zeros([seg_num, crack_positions_num])
# F_s = np.zeros([seg_num, crack_positions_num])
# F_a = np.zeros([seg_num, crack_positions_num])
# slip = np.zeros([seg_num, crack_positions_num])
# steel_slip = np.zeros([seg_num, crack_positions_num])
# shear_stress = np.zeros([seg_num, crack_positions_num])
# w = np.zeros([seg_num, crack_positions_num])
# w_steel = np.zeros([seg_num, crack_positions_num])
# F_agg = np.zeros([seg_num, crack_positions_num])
# for i in range(len(seg_)):
#     ce = CrackPropagation(n_seg=seg_[i])
#     for j in range(len(crack_positions)):
#         ce.reset()
#         ce.sz_cp.trait_set(x_00=crack_positions[i])
#         print(crack_positions[i])
#         ce.sz_cp.sz_ctr.trait_set(L_fps=80)
#         #print(L_fps_[j])
#         ce.run()
#         F_beam = ce.F_beam
#         F_max_ = np.max(F_beam)
#         F_max_i_ = np.argmax(F_beam)
#         F_max_i_int = np.array(F_max_i_)
#         F_Na_ = ce.F_Na[F_max_i_int]
#         F_s_ = ce.F_s[F_max_i_int]
#         slip_ = ce.slip[F_max_i_int][0]
#         w_ = ce.w[F_max_i_int][0]
#         steel_slip_ = ce.s_steel[F_max_i_int]
#         w_steel_ = ce.w_steel[F_max_i_int]
#         shear_stress_ = ce.shear_agg[F_max_i_int][0]
#         F_a_ = ce.F_a[F_max_i_int][1]
#         F_agg_ = F_a_ - F_Na_
#         F_max[i, j] = F_max_
#         F_Na[i,j] = F_Na_
#         F_s[i,j] = F_s_
#         slip[i,j] = slip_
#         w[i,j] = w_
#         steel_slip[i,j] = steel_slip_
#         w_steel[i,j] = w_steel_
#         shear_stress[i,j] = shear_stress_
#         F_a[i,j] = F_a_
#         F_agg[i,j] = F_agg_
        
        
# #         F_max_i.append(F_max_i_)
#     #F_max_all = np.concatenate(F_max_1)
# print('F_max =', F_max)
# print('F_Na =', F_Na)
# print('F_s =', F_s)
# print('slip =', slip)
# print('w =', w)
# print('steel_slip =', steel_slip)
# print('w_steel =', w_steel)
# print('shear_stress =', shear_stress)
# print('F_a =', F_a)
#     #r.append(F_max)
# #         F_Na.append(F_Na_)
# #         F_s.append(F_s_)
# #         F_a.append(F_a_)
# #         F_agg.append(F_agg_)
# #         #slip.append(slip_)
# #         #w.append(w_)
# #         steel_slip.append(steel_slip_)
# #         w_steel.append(w_steel_)
#         #shear_stress.append(shear_stress_)
#     #print('slip =', slip)
#     #print('w =', w)
# #     print('steel_slip =', steel_slip)
# #     print('L_fps =', L_fps_)
# #     print('crack_position =', crack_positions)

In [ ]:
# import numpy as np
# crack_positions = np.linspace(0.1 * ce.sz_bd.L, 0.9 * ce.sz_bd.L, 10)
# F_max = []
# F_max_i = []
# slip = []
# F_Na = []
# shear_stress = []
# for crack_position in crack_positions:
#     ce.reset()
#     ce.sz_cp.trait_set(x_00=crack_position)
#     ce.run()
#     F_beam = ce.F_beam
#     F_max_ = np.max(F_beam)
#     F_max_i_ = np.argmax(F_beam)
#     F_max_i_int = np.array(F_max_i_)
#     F_Na_ = ce.F_Na[F_max_i_int]
#     slip_ = ce.slip[F_max_i_int][F_max_i_int]
#     shear_stress_ = ce.shear_agg[F_max_i_int][F_max_i_int]
#     F_max.append(F_max_)
#     F_max_i.append(F_max_i_)
#     slip.append(slip_)
#     shear_stress.append(shear_stress_)
#     F_Na.append(F_Na_)
#     F_max_i.append(F_max_i_)
# print('crack position =', crack_positions)
# print('slip =', slip)
# print('shear stress =', shear_stress)
# print('F_Na =', F_Na)
# print('F_max_i =', F_max_i)

In [ ]:
# import matplotlib.pylab as plt
# _, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
# #ax1.plot(crack_positions, np.array(V_c) /1000, color = 'red', lw= 2);
# ax1.plot(crack_positions, np.array(F_a) , color = 'blue', lw= 2, label = '$F_{agg}$'); #/1000
# ax1.plot(crack_positions, F_Na, color = 'red', lw = 2, label = '$F_{dowel}$');
# #ax1.plot(crack_positions, np.array(Q) , color = 'green', lw= 2); #/1000
# ax1.plot(crack_positions, np.array(Q_red) , color = 'orange', lw= 2, label = '$F_{uncr}$'); #/1000
# ax1.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax1.set_ylabel(r'$F\,(N)$');
# ax1.set_title(r'Max Force Contribution')
# ax1.legend()
# ax2.plot(crack_positions, np.array(F_a) , color = 'blue', lw= 2, label = '$F_{agg}$');
# ax2.plot(crack_positions, F_Na, color = 'red', lw = 2, label = '$F_{dowel}$');
# ax2.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax2.set_ylabel(r'$F\,(N)$');
# #plt.savefig("D:\Shear zones\Python plots\Force_comparisons_updated.pdf")

In [ ]:
# import matplotlib.pylab as plt
# _, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
# ax1.plot(c_p, F_La, color = 'red', lw= 2);
# ax1.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax1.set_ylabel(r'$F\,(N)$');
# ax1.set_title(r'Force contribution Aggregate Interlock')
# ax2.plot(c_p , F_Na, color='blue', lw=2);
# ax2.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax2.set_ylabel(r'$F_{da}\,(N)$');
# ax2.set_title(r'Force contribution Dowel Action')
# #plt.savefig("D:\Shear zones\Python plots\Force_comparisons.pdf")

In [ ]:
# import numpy as np
# F_a_ = np.array(ce.F_a)
# D = ce.sz_bd.H - ce.sz_bd.cross_section_layout.reinforcement.z_j[0]
# x_tip_1n = np.array(ce.x_tip_1n)
# x_tip_0n = np.array(ce.x_tip_0n)
# x_tip_1k = np.array(ce.x_tip_1k)
# L = ce.sz_bd.L
# A_s = ce.sz_bd.cross_section_layout.reinforcement.A_j[0] 
# E_s = ce.sz_bd.cross_section_layout.reinforcement.E_j[0]
# v_n = np.array(ce.v_n)
# V_c = []
# # for x_tip_0n in range(len(v_n)):
# #     if x_tip_0n == 0:
# #         V_c = ((ce.crack_tip_shear_stress.Q) ) #\
# #                #/ (ce.sz_bd.B * D * np.sqrt(33.3) ))
# #     elif x_tip_0n > 0:
# V_c = (np.array(ce.F_beam[1:])) / (1 - (0.5 * (ce.sz_bd.H - x_tip_1n[1:])/ x_tip_0n[1:])) #\ #ce.crack_tip_shear_stress.Q
#                #/ (ce.sz_bd.B * D * np.sqrt(33.3) ))      
# #V_c = ((ce.crack_tip_shear_stress.Q) / (1 - (0.5 * (ce.sz_bd.H - x_tip_1n[1:])/ x_tip_0n[1:])) / (ce.sz_bd.B * D * np.sqrt(33.3) ))
# V_e = np.ones_like(v_n) * 114000
# V_f = v_n * A_s * E_s * (D - x_tip_1k/3) / ((D - x_tip_1k) * (L - x_tip_0n))
# #V_f
# #v_n
# #ce.Q
# #V_c

In [ ]:
# import matplotlib.pylab as plt
# _, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,4))
# ax1.plot(ce.v_n[1:], V_c, color = 'red', lw= 2, label = 'V_c');
# #ax1.plot(ce.v_n, ce.F_beam, color = 'blue', lw = 2);
# ax1.plot(ce.v_n[1:], V_e[1:], '--' , color = 'orange', lw = 2, label = 'V_exp')
# ax1.plot(ce.v_n[1:], V_f[1:], '--' , color = 'green', lw = 2, label = 'V_load')
# ax1.set_xlabel('deflection')
# ax1.set_ylabel('V')
# ax1.legend()

In [ ]:
# import matplotlib.pylab as plt
# _, (ax1) = plt.subplots(1, 1, figsize=(7,4))
# ax1.plot(c_p, F_m, color = 'red', lw= 2);
# ax1.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax1.set_ylabel(r'$F_{max}\,(N)$');
# ax1.set_title(r'Maximum Force Contribution along the length of the beam')
# #plt.savefig("D:\Shear zones\Python plots\Max_Force_contributions.pdf")

In [ ]:
# import matplotlib.pylab as plt
# _, (ax1) = plt.subplots(1, 1, figsize=(7,5))
# x_00_L = c_p/ ce.sz_bd.L
# ones = np.ones_like(c_p)
# V_exp = 114000 * ones 
# ax1.plot(x_00_L, F_m, color = 'red', lw= 2, label = (r'$V_{max}$'));
# ax1.plot(x_00_L, V_exp, color = 'black',linestyle='--', lw =2, label = (r'$V_{exp}$'));
# ax1.set_xlabel(r'$\mathrm{\frac{x_{00}}{Length\,of\,the\,beam\,(mm)}}$'); ax1.set_ylabel(r'$F_{max}\,(N)$');
# ax1.set_title(r'Maximum Force Contribution along the length of the beam')
# ax1.legend()
# #plt.savefig("D:\Shear zones\Python plots\Max_Force_contributions_with_V_exp.pdf")

In [ ]:
# import matplotlib.pylab as plt
# _, (ax1) = plt.subplots(1, 1, figsize=(6,5))
# ones = np.ones_like(c_p)
# V_exp = 114000 * ones 
# #ax1.plot(c_p, F_m, color = 'red', lw= 2, label = (r'$V_{max}$'));
# #ax1.plot(c_p, V_exp, color = 'black',linestyle='--', lw =2, label = (r'$V_{exp}$'));
# ax1.plot(c_p, Q_reduce, color = 'blue', lw =2, label = (r'$V_{uncr}$'));
# #ax1.plot(c_p, F_La, color = 'orange', lw = 2, label = (r'$V_{agg}$'));
# #ax1.plot(c_p, F_Na, color = 'green', lw =2, label = (r'$V_{da}$'));
# ax1.set_xlabel(r'$\mathrm{Length\,of\,the\,beam\,(mm)}$'); ax1.set_ylabel(r'$F_{max}\,(N)$');
# ax1.set_title(r'Maximum Force Contribution along the length of the beam')
# ax1.legend()
# plt.savefig("D:\Shear zones\Python plots\Q_reduce_contributions.pdf")

In [ ]:
# ce.crack_tip_orientation.get_psi(), ce.M_n

In [ ]:
# ce.crack_tip_shear_stress.Q_reduced, ce.sz_stress_profile.F_La[47,0], ce.sz_stress_profile.F_Na[0,0]
# ce.crack_tip_shear_stress.tau_x_tip_1k *ce.sz_bd.B
# ce.sz_stress_profile.F_a
# ce.crack_tip_shear_stress.Q_reduced

In [ ]:
# import numpy as np
# ce.sz_stress_profile.u_La[47,:]

In [ ]:
# (ce.sz_stress_profile.M / 1000) / ((ce.crack_tip_shear_stress.Q) *\
#                                    (ce.sz_bd.H - ce.sz_bd.cross_section_layout.reinforcement.z_j[0]))

In [ ]:
# #V_uncr = np.sum(ce.crack_tip_shear_stress.tau_z) * ce.sz_bd.B
# print(V_uncr)
# V_da = ce.sz_stress_profile.F_Na[:,1]
# print(V_da)
# V_ai = np.sum(ce.sz_stress_profile.F_La[:,1]) #* ce.sz_bd.B #- np.sum(ce.sz_bd.cmm.get_sigma_ag(ce.sz_stress_profile.u_Lb[...,0],ce.sz_stress_profile.u_Lb[...,1])) #* np.cos((ce.sz_cp._get_beta()))
# print(V_ai)
# V_res = np.max(ce.F_beam)
# print(V_res)
# V_uncr + V_da + V_ai + V_res

In [ ]:
# np.sum(ce.sz_stress_profile.F_La[:,1])

In [ ]:
# np.sum(ce.sz_stress_profile.S_La[:,1]) * ce.sz_bd.B

In [ ]:
# import numpy as np
# S_Lb = ce.sz_stress_profile.S_Lb
# S_La = ce.sz_stress_profile.S_La
# idx = np.argmax(S_Lb[...,0])

In [ ]:
# S_Lb[idx,:], S_La[idx,:]

In [ ]:
# ce.sz_cp.T_Mab[idx,...]

In [ ]:
# np.einsum('i,i', S_Lb[idx,:], S_Lb[idx,:])

In [ ]:
# np.einsum('i,i', S_La[idx,:], S_La[idx,:])

Todo:

 - include the reinforcement material law - partially done 
   (but for strain-based material law) - replace with the crack bridge law
 - transformation of the crack bridging force to the global coordinates
 - include the dowel effect 

In [ ]:
#ce.sz_ctr._get_x_rot_ak()

In [ ]:
# ce.sz_stress_profile.F_Na

In [ ]:
# ce.sz_stress_profile.u_Na

In [ ]:
# np.max(ce.sz_stress_profile.F_La)

In [ ]:
# np.sum(ce.sz_stress_profile.F_La)

In [ ]:
# np.max(ce.sz_stress_profile.F_La)

In [21]:
import sympy as sp
import matplotlib.pylab as plt
w,f_c, s = sp.symbols('w,f_c,s')
tau_s_wal = sp.Piecewise(
        (0, w <= 0),
        ((- f_c / 30) + (1.8 * w**(-0.8) + (0.234 * w**(-0.707) - 0.2) * f_c ) * s, w > 0)
    )

f_c_ = 33.3
get_tau_s_wal = sp.lambdify((w,s), tau_s_wal.subs(f_c,f_c_), 'numpy')

w_min = 0 #-1
w_max = 3
w_data = np.linspace(w_min, w_max, 100)
s_max = 3
s_data = np.linspace(0*s_max, 1.1*s_max, 100) #-1.1
s_, w_ = np.meshgrid(s_data, w_data)
fig, ax = plt.subplots(1,1,figsize=(10,4))
ax_3d = fig.add_subplot(1, 2, 1, projection='3d')
tau_s = get_tau_s_wal(w_, s_)
ax_3d.plot_surface(w_, s_, tau_s, cmap='viridis', edgecolor='none')
ax_3d.set_xlabel(r'$w\;\;\mathrm{[mm]}$', fontsize=12)
ax_3d.set_ylabel(r'$s\;\;\mathrm{[mm]}$', fontsize=12)
ax_3d.set_zlabel(r'$\tau\;\;\mathrm{[MPa]}$', fontsize=12)
ax_3d.set_title('aggregate interlock law', fontsize=12)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<lambdifygenerated-49>:2: RuntimeWarning: divide by zero encountered in power
  return (select([less_equal(w, 0),True], [0,s*(1.8*w**(-0.8) + 7.7922*w**(-0.707) - 6.66) - 1.11], default=nan))
<lambdifygenerated-49>:2: RuntimeWarning: invalid value encountered in multiply
  return (select([less_equal(w, 0),True], [0,s*(1.8*w**(-0.8) + 7.7922*w**(-0.707) - 6.66) - 1.11], default=nan))


Text(0.5, 0.92, 'aggregate interlock law')

In [20]:
get_tau_s_wal

<function _lambdifygenerated(w, s)>